In [51]:
import pandas as pd
from pydantic import BaseModel
import pprint

from openai import OpenAI
client = OpenAI()

In [170]:
def read_parquet(uploaded_file):
    return pd.read_parquet(uploaded_file)

# toutes les données 1 an sur 3 chaines 
df = read_parquet("/Users/anne/Desktop/3_channels_predictions_09_2023_09_2024.parquet")
df.reset_index(inplace=True)
df_claims = df.explode('claims', ignore_index=True)
df_claims = pd.json_normalize(df_claims['claims'])
df_claims = df_claims[(df_claims['disinformation_score']=='high') & (df_claims['pro_anti']!='pro-écologie')]

In [113]:
df

,id,start,text,channel_name,channel_is_radio,channel_program_type,channel_program,themes,keywords,num_keywords,num_tokens,claims
0,1dcd4b454f8bac42440259ce26a1a2192051186bb5728b...,2023-09-01 06:26:00,<unk> <unk> <unk> <unk> aerosmith en janvier m...,europe1,True,Information - Magazine,Bonjour,"[""biodiversite_concepts_generaux"", ""adaptation...","[{""keyword"": ""eaux"", ""timestamp"": 169354246806...",1,383,[{'analysis': 'L'allégation semble promouvoir ...
1,0eb5805fa23e0819f817ea10fe1fccd19e61e40a1239cc...,2023-09-01 06:50:00,la très grande majorité d'entre eux ne connais...,europe1,True,Information - Magazine,Bonjour,"[""biodiversite_concepts_generaux"", ""changement...","[{""keyword"": ""eaux"", ""timestamp"": 169354380309...",3,593,[{'analysis': 'Cette affirmation est généralem...
2,b6d54aefb250671e7754a688411ce9e68badcad88a665b...,2023-09-01 07:38:00,mais titeuf ne vieillit pas le monde change ti...,europe1,True,Information - Magazine,Europe 1 Matin,"[""changement_climatique_constat""]","[{""keyword"": ""bilan carbone"", ""timestamp"": 169...",1,606,[{'analysis': 'L'allégation sur le fait que Ti...
3,23c2d3b292d9ab0fb0d0b2b8c34f3b88708c79ffab2d66...,2023-09-01 08:44:00,dû travailler très vite le journal arrive à no...,europe1,True,Information - Magazine,Europe 1 Matin,"[""biodiversite_concepts_generaux"", ""ressources...","[{""keyword"": ""eaux"", ""timestamp"": 169355074908...",3,515,[{'analysis': 'L'affirmation que le nucléaire ...
4,71df0ce2b34afa23391d8e31d35ccd213ae2a881b7ce41...,2023-09-01 08:46:00,pas avoir lieu ni même européens existait ni l...,europe1,True,Information - Magazine,Europe 1 Matin,"[""biodiversite_concepts_generaux"", ""changement...","[{""keyword"": ""\u00e9cologiste"", ""timestamp"": 1...",2,536,[{'analysis': 'Cette allégation souligne un év...
...,...,...,...,...,...,...,...,...,...,...,...,...
34514,121c2c2780f373e0d03ad266a5b4c11e87873e35395405...,2024-09-12 07:06:00,est la période la plus à risque pour les inond...,sud-radio,True,Information - Magazine,Le Grand Matin,"[""adaptation_climatique_solutions"", ""changemen...","[{""keyword"": ""inondation"", ""timestamp"": 172611...",2,575,[{'analysis': 'Cette déclaration peut être con...
34515,10e4d16cad946340c67738bdaa487603d4635c6d2db978...,2024-09-12 08:00:00,solaire point fr votre expert en panneaux sola...,sud-radio,True,Information - Magazine,Le Grand Matin,"[""ressources_solutions"", ""changement_climatiqu...","[{""keyword"": ""panneaux solaires"", ""timestamp"":...",1,526,[{'analysis': 'Cette allégation est factuelle ...
34516,74f1b824b772877e6e05908a308a8ae31146b652ff5079...,2024-09-12 09:00:00,foyer trois mille neuf cent quatre-vingt-dix e...,sud-radio,True,Information - Magazine,Le Grand Matin,"[""ressources_solutions"", ""changement_climatiqu...","[{""keyword"": ""panneaux solaires"", ""timestamp"":...",1,520,[{'analysis': 'Cette allégation est généraleme...
34517,dbb0ba82809710c830b2170abb1d30b4b0b07650b3fdad...,2024-09-12 11:48:00,vous aimeriez tester tous les milieux b quatre...,sud-radio,True,Information - Magazine,Mettez-vous d'accord,"[""biodiversite_solutions_indirectes"", ""changem...","[{""keyword"": ""panneaux solaires"", ""timestamp"":...",1,517,[]


In [225]:
# classification CARDS
from openai import AsyncOpenAI
import asyncio
from tqdm.asyncio import tqdm
import tiktoken

model = "gpt-4o-mini"
encoding = tiktoken.encoding_for_model(model)

system_prompt = f"""
Tu es expert en désinformation sur les sujets environnementaux, expert en science climatique et sachant tout sur le GIEC. 
Je vais te donner des claims sur des thématiques environnementales extraits de retranscriptions d'un flux TV ou Radio à factchecker. 
Pour chaque claim, donne une catégorisation de cette allégation suivant la classification qui suit.
Renvoie le résultat en json sans autre phrase d'introduction ou de conclusion avec à chaque fois le champ suivant : "cards_label_pred"

<catégories prédéfinies>
- 0_accepted: Lorsque l'ensemble de l'extrait est neutre ou reflète le consensus scientifique sur les questions climatiques, sans contenir de désinformation. Cette catégorie est aussi adaptée si l'extrait n'aborde pas le sujet du climat.
- 1_its_not_happening: Le réchauffement climatique n'a pas lieu. Cette catégorie soutient que le changement climatique est inexistant ou largement exagéré. Elle suggère que des facteurs naturels — tels que le froid, la préservation de la glace et le refroidissement des océans — prouvent que le réchauffement n'est pas en cours. De plus, elle implique que les prévisions de changements climatiques sévères, y compris la montée du niveau de la mer et les phénomènes météorologiques extrêmes, sont exagérées ou inventées. Certaines affirmations soutiennent que la terminologie climatique a été manipulée (par exemple, en passant de "réchauffement climatique" à "changement climatique") pour alarmer le public sans raison.
- 2_its_not_us: Les humains ne sont pas à l'origine du réchauffement climatique. Cette catégorie avance que les activités humaines, en particulier les émissions de gaz à effet de serre, ne sont pas responsables du réchauffement observé. Elle soutient que les cycles naturels, l'activité solaire ou d'autres facteurs non humains sont à l'origine des changements climatiques. Les affirmations de cette catégorie minimisent souvent le rôle du CO2, ou soulignent que les changements climatiques passés se sont produits sans intervention humaine, suggérant que le réchauffement actuel est également naturel.
- 3_its_not_bad: Les impacts climatiques ne sont pas mauvais. Cette catégorie reconnaît qu'un certain réchauffement pourrait se produire, mais affirme qu'il pose peu de risques. Les arguments de cette catégorie suggèrent que la sensibilité du climat au CO2 est faible, ce qui limiterait le réchauffement. Les partisans prétendent que l'environnement, les espèces et la santé humaine seront largement épargnés, car les plantes bénéficient de l'augmentation du CO2, et que de nombreuses espèces, comme les ours polaires, peuvent s'adapter. Dans l'ensemble, elle suggère que les impacts climatiques seront mineurs et gérables.
- 4_solutions_wont_work: Les solutions climatiques ne fonctionneront pas. Cette catégorie s'oppose aux politiques climatiques proposées, en affirmant qu'elles seront nuisibles économiquement ou socialement, inefficaces ou irréalistes. Elle soutient que les politiques de réduction des émissions augmenteront les coûts, affaibliront la sécurité nationale et restreindront les libertés individuelles sans bénéfices climatiques significatifs. D'autres affirmations proposent que les solutions "vertes" sont peu fiables, que l'énergie propre est non éprouvée, ou que les combustibles fossiles sont abondants et essentiels pour l'avenir. Essentiellement, elle soutient que les politiques climatiques sont trop coûteuses, difficiles ou inefficaces pour être poursuivies.
- 5_science_is_unreliable: Les mouvements pour le climat et la science du climat ne sont pas fiables. Cette catégorie attaque la crédibilité de la science climatique et du mouvement pour le climat, affirmant qu'ils sont non fiables, biaisés ou idéologiquement orientés. Elle remet en question le consensus scientifique sur le changement climatique, suggérant que les données, modèles et méthodes sont défectueux. De plus, elle présente le mouvement climatique comme alarmiste, biaisé ou faisant partie d'une conspiration plus large, motivée par des intentions politiques ou religieuses plutôt qu'une véritable préoccupation. Ce cadre cherche à miner la confiance dans la science climatique et dans ceux qui plaident pour l'action.
</catégories prédéfinies>
    """
prompt_tokens = len(encoding.encode(system_prompt))
    
async def classify_claims(transcription, system_prompt: str = system_prompt, prompt_tokens: str = prompt_tokens, tpm_limit: int = 2e6):

    text_tokens = len(encoding.encode(transcription))
    total_tokens = prompt_tokens + text_tokens
    wait_time_s = 60 * total_tokens / tpm_limit
    await asyncio.sleep(wait_time_s)

    response = await client.beta.chat.completions.parse(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": transcription},
        ],
        temperature=0,
    )
    claims_data = response.choices[0].message.parsed
    #result = pd.DataFrame([claim.dict() for claim in claims_data.claims])
    
    return claims_data #esult

async def run_classification(texts: list[str]):
    semaphore = asyncio.Semaphore(10)
    
    async def bounded_classify_claims(text):
        async with semaphore:
            return await classify_claims(text)
    
    classification = await tqdm.gather(*[bounded_classify_claims(text) for text in texts])
    return classification

In [226]:
classify_claims(['les écolos veulent nous enlever nos libertés'])

<coroutine object classify_claims at 0x12c54ae60>

In [ ]:
tqdm.gather(*[bounded_classify_claims(text) for text in ['les écolos veulent nous enlever nos libertés']])

In [213]:
result = await run_classification(['les écolos veulent nous enlever nos libertés'])

  0%|          | 0/1 [00:00<?, ?it/s]


TypeError: object ParsedChatCompletion[NoneType] can't be used in 'await' expression

In [199]:
claims = df_claims.iloc[:20]['claim']
claims

52           les écolos veulent nous enlever nos libertés
71      pas d'interdiction d'utilisation de produits q...
133     le moustique tigre engendre des coûts estimés ...
159     Les écologistes vont devoir accepter l'idée qu...
185     Manger de la viande utilise moins d'eau que le...
188     Les écologistes imposent des politiques de man...
333     Le manque d'hygiène des personnes a contribué ...
429     Les climatologues ne croient plus leurs termes...
505     Le séisme au Maroc peut être causé par le chan...
564     Ce tremblement de terre a été causé par des ch...
858     les migrants sont à l'origine du sel fin du Bu...
897     L'effondrement climatique signifie que notre c...
973     Le phénomène des migrations climatiques n'est ...
988     beaucoup de soldats poneys menteurs à tête de ...
997          Il y a une pluie d'acide mortelle imminente.
1118       les loups deviennent dangereux pour les hommes
1156    Ces mesures écologiques interdisent aux gens d...
1158    L'inve

In [193]:
claims.dtypes

dtype('O')

In [202]:
df_claims.iloc[:1]['claim'].values

array(['les écolos veulent nous enlever nos libertés'], dtype=object)

In [182]:
result.content

AttributeError: 'coroutine' object has no attribute 'content'

In [178]:
claims.values

array(['les écolos veulent nous enlever nos libertés',
       "pas d'interdiction d'utilisation de produits qui nous permettent protéger nos plantes nos animaux tant que nous n'avons pas de solution alternative qui fonctionne",
       "le moustique tigre engendre des coûts estimés à quatre cent cinquante-six milliards d'euros par an",
       "Les écologistes vont devoir accepter l'idée qu'il faille éradiquer, y compris par des insecticides, des espèces vivantes animales ou végétales.",
       "Manger de la viande utilise moins d'eau que les alternatives végétales.",
       'Les écologistes imposent des politiques de manière dictatoriale.',
       "Le manque d'hygiène des personnes a contribué à l'infestation des punaises de lit.",
       'Les climatologues ne croient plus leurs termes honnêtes.',
       'Le séisme au Maroc peut être causé par le changement climatique.',
       'Ce tremblement de terre a été causé par des changements climatiques.',
       "les migrants sont à l'origine 

In [220]:
# classification CARDS
class Claim(BaseModel):
    cards_label_pred: str

class Claims(BaseModel):
    claims: list[Claim]

def classify_claims(transcription):
    prompt = f"""
Tu es expert en désinformation sur les sujets environnementaux, expert en science climatique et sachant tout sur le GIEC. 
Je vais te donner des claims sur des thématiques environnementales extraits de retranscriptions d'un flux TV ou Radio à factchecker. 
Pour chaque claim, donne une catégorisation de cette allégation suivant la classification qui suit.
Renvoie le résultat en json sans autre phrase d'introduction ou de conclusion avec à chaque fois le champ suivant : "cards_label_pred"

<catégories prédéfinies>
- 0_accepted: Lorsque l'ensemble de l'extrait est neutre ou reflète le consensus scientifique sur les questions climatiques, sans contenir de désinformation. Cette catégorie est aussi adaptée si l'extrait n'aborde pas le sujet du climat.
- 1_its_not_happening: Le réchauffement climatique n'a pas lieu. Cette catégorie soutient que le changement climatique est inexistant ou largement exagéré. Elle suggère que des facteurs naturels — tels que le froid, la préservation de la glace et le refroidissement des océans — prouvent que le réchauffement n'est pas en cours. De plus, elle implique que les prévisions de changements climatiques sévères, y compris la montée du niveau de la mer et les phénomènes météorologiques extrêmes, sont exagérées ou inventées. Certaines affirmations soutiennent que la terminologie climatique a été manipulée (par exemple, en passant de "réchauffement climatique" à "changement climatique") pour alarmer le public sans raison.
- 2_its_not_us: Les humains ne sont pas à l'origine du réchauffement climatique. Cette catégorie avance que les activités humaines, en particulier les émissions de gaz à effet de serre, ne sont pas responsables du réchauffement observé. Elle soutient que les cycles naturels, l'activité solaire ou d'autres facteurs non humains sont à l'origine des changements climatiques. Les affirmations de cette catégorie minimisent souvent le rôle du CO2, ou soulignent que les changements climatiques passés se sont produits sans intervention humaine, suggérant que le réchauffement actuel est également naturel.
- 3_its_not_bad: Les impacts climatiques ne sont pas mauvais. Cette catégorie reconnaît qu'un certain réchauffement pourrait se produire, mais affirme qu'il pose peu de risques. Les arguments de cette catégorie suggèrent que la sensibilité du climat au CO2 est faible, ce qui limiterait le réchauffement. Les partisans prétendent que l'environnement, les espèces et la santé humaine seront largement épargnés, car les plantes bénéficient de l'augmentation du CO2, et que de nombreuses espèces, comme les ours polaires, peuvent s'adapter. Dans l'ensemble, elle suggère que les impacts climatiques seront mineurs et gérables.
- 4_solutions_wont_work: Les solutions climatiques ne fonctionneront pas. Cette catégorie s'oppose aux politiques climatiques proposées, en affirmant qu'elles seront nuisibles économiquement ou socialement, inefficaces ou irréalistes. Elle soutient que les politiques de réduction des émissions augmenteront les coûts, affaibliront la sécurité nationale et restreindront les libertés individuelles sans bénéfices climatiques significatifs. D'autres affirmations proposent que les solutions "vertes" sont peu fiables, que l'énergie propre est non éprouvée, ou que les combustibles fossiles sont abondants et essentiels pour l'avenir. Essentiellement, elle soutient que les politiques climatiques sont trop coûteuses, difficiles ou inefficaces pour être poursuivies.
- 5_science_is_unreliable: Les mouvements pour le climat et la science du climat ne sont pas fiables. Cette catégorie attaque la crédibilité de la science climatique et du mouvement pour le climat, affirmant qu'ils sont non fiables, biaisés ou idéologiquement orientés. Elle remet en question le consensus scientifique sur le changement climatique, suggérant que les données, modèles et méthodes sont défectueux. De plus, elle présente le mouvement climatique comme alarmiste, biaisé ou faisant partie d'une conspiration plus large, motivée par des intentions politiques ou religieuses plutôt qu'une véritable préoccupation. Ce cadre cherche à miner la confiance dans la science climatique et dans ceux qui plaident pour l'action.
</catégories prédéfinies>

<transcription>
{transcription}
</transcription>
    """

    completion = client.beta.chat.completions.parse(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        response_format=Claims,
    )
    n_tokens = completion.usage.total_tokens
    claims_data = completion.choices[0].message.parsed
    result = pd.DataFrame([claim.dict() for claim in claims_data.claims])

    return claims_data #result, n_tokens

In [218]:
result = classify_claims(['les écolos veulent nous enlever nos libertés'])

In [221]:
result

Claims(claims=[Claim(cards_label_pred='4_solutions_wont_work')])

In [162]:
df_claims

,analysis,claim,context,contradiction,disinformation_category,disinformation_score,pro_anti,quote,speaker
52,Cette allégation semble perpétuer un narratif ...,les écolos veulent nous enlever nos libertés,"Dans un segment de l'émission, un intervenant ...",N/A,narrative,high,anti-écologie,les écolos veulent nous enlever nos libertés,N/A
71,Cette position pourrait mener à un statu quo s...,pas d'interdiction d'utilisation de produits q...,L'intervenant revendique qu'il ne devrait pas ...,N/A,consensus,high,anti-écologie,pas d'interdiction d'utilisation de produits q...,N/A
133,Cette affirmation semble exagérée ou nécessite...,le moustique tigre engendre des coûts estimés ...,"En parlant du moustique tigre, le locuteur évo...",N/A,facts,high,anti-écologie,la facture est très salée un c à un coût touss...,N/A
159,Cette assertion pourrait être considérée comme...,Les écologistes vont devoir accepter l'idée qu...,Jean François Pécresse a évoqué que les écolog...,N/A,narrative,high,anti-écologie,ce rapport va aussi demander aux écologistes d...,Jean François Pécresse
185,Cette affirmation est généralement contraire a...,Manger de la viande utilise moins d'eau que le...,L'intervenant évoque les effets de la consomma...,N/A,facts,high,anti-écologie,manger de la viande... deux moins d'eau.,N/A
...,...,...,...,...,...,...,...,...,...
63370,Bien que l'expression 'trop plein d'eau' soit ...,Les mauvais résultats des récoltes ne sont pas...,L'intervenant affirme que les difficultés de r...,N/A,consensus,high,anti-écologie,Ces mauvais résultats y sont pas dus à la cani...,Bertrand Bartolini
63379,Cette affirmation présente un chiffre qui semb...,"Pour l'électrique sur l'autoroute, on est plut...",Ehsan Emami évoque les coûts associés à la rec...,N/A,facts,high,anti-écologie,"Pour l'électrique sur l'autoroute, on est plut...","Ehsan Emami, PDG de Koh Voltis"
63694,Cette allégation semble exploiter des comparai...,les nazis et hitler le nazisme sont de la biol...,"L'intervenant évoque un lien entre le nazisme,...",N/A,narrative,high,anti-écologie,que les nazis et hitler le nazisme et de la bi...,N/A
63701,Cette affirmation nie le consensus scientifiqu...,Le changement climatique est un canular.,Cette allégation a été faite dans un contexte ...,N/A,consensus,high,anti-écologie,le changement climatique qu'est un canular.,N/A


In [172]:
result, n_tokens = classify_claims(df_claims['claim'])

In [169]:
df_claims['claim']

52            les écolos veulent nous enlever nos libertés
71       pas d'interdiction d'utilisation de produits q...
133      le moustique tigre engendre des coûts estimés ...
159      Les écologistes vont devoir accepter l'idée qu...
185      Manger de la viande utilise moins d'eau que le...
                               ...                        
63370    Les mauvais résultats des récoltes ne sont pas...
63379    Pour l'électrique sur l'autoroute, on est plut...
63694    les nazis et hitler le nazisme sont de la biol...
63701             Le changement climatique est un canular.
63703        Les migrants mangent des chiens et des chats.
Name: claim, Length: 1586, dtype: object

In [173]:
result.shape

(9, 1)

In [174]:
result

,cards_label_pred
0,4_solutions_wont_work
1,0_accepted
2,0_accepted
3,0_accepted
4,3_its_not_bad
5,0_accepted
6,1_its_not_happening
7,0_accepted
8,1_its_not_happening


In [145]:
def disinformation_score(row):
    try:
        return 'high' in [row.claims[i]['disinformation_score'] for i in range(len(row))]
    except:
        return [False]

score = df.apply(lambda row: disinformation_score(row), axis=1)

0        [{'analysis': 'L'allégation semble promouvoir ...
1        [{'analysis': 'Cette affirmation est généralem...
2        [{'analysis': 'L'allégation sur le fait que Ti...
3        [{'analysis': 'L'affirmation que le nucléaire ...
4        [{'analysis': 'Cette allégation souligne un év...
                               ...                        
34514    [{'analysis': 'Cette déclaration peut être con...
34515    [{'analysis': 'Cette allégation est factuelle ...
34516    [{'analysis': 'Cette allégation est généraleme...
34517                                                   []
34518                                                   []
Name: claims, Length: 34519, dtype: object

In [148]:
df['claims'].iloc[0]

array([{'analysis': "L'allégation semble promouvoir des pratiques positives de conservation de l'eau, ce qui est généralement en accord avec les objectifs écologiques. Cependant, il serait pertinent de vérifier les spécificités de ces initiatives et leur efficacité.", 'claim': "les initiatives positives de jean z une nova sion pour économiser l'eau", 'context': "L'interview mentionne des initiatives sur la gestion de l'eau, particulièrement celles visant à encourager des pratiques d'économie d'eau dans des contextes comme le camping.", 'contradiction': 'N/A', 'disinformation_category': 'facts', 'disinformation_score': 'low', 'pro_anti': 'pro-écologie', 'quote': "les initiatives positives de jean z une nova sion pour économiser l'eau", 'speaker': 'Emmanuel Kessler, directeur de la rédaction du magazine Capital'}],
      dtype=object)

In [160]:
df_claims.groupby('pro_anti').count()

,analysis,claim,context,contradiction,disinformation_category,disinformation_score,quote,speaker
pro_anti,,,,,,,,
Anti-écologie,6,6,6,6,6,6,6,6
N/A,9,9,9,9,9,9,9,9
anti,1,1,1,1,1,1,1,1
anti-écologie,1567,1567,1567,1567,1567,1567,1567,1567
anti-écologie (dans le sens où cela décontextualise les risques de manière alarmante),1,1,1,1,1,1,1,1
anti-écologie (en critiquant l'impact des idéologies écologiques),1,1,1,1,1,1,1,1
anti-écologie (en induisant un jugement négatif sur des groupes de personnes),1,1,1,1,1,1,1,1


In [157]:
df_claims[df_claims['pro_anti']=='pro-écologie']

,analysis,claim,context,contradiction,disinformation_category,disinformation_score,pro_anti,quote,speaker
350,Cet argument est soutenu par des travaux scien...,La fonte de la glace en Antarctique peut entra...,Le climatologue Mathieu Casado explique que l'...,N/A,facts,high,pro-écologie,Cela peut avoir des conséquences potentielles ...,"Mathieu Casado, climatologue au CNRS"
1126,Bien que la surmortalité des poissons puisse r...,La surmortalité des poissons est due à des déc...,Mention d'un incident où des militants écologi...,N/A,consensus,high,pro-écologie,le maire dénonce une surmortalité de poisson,N/A
2198,Cette affirmation nécessite une vérification. ...,Notre bilan carbone 2023 va avoir une réductio...,Un transporteur routier annonce une réduction ...,N/A,facts,high,pro-écologie,Notre bilan carbone 2023 va avoir une réductio...,N/A
2556,Cette affirmation est contestée par de nombreu...,Le glyphosate n'a pas de résidus dans les cham...,La personne affirme que si le glyphosate est u...,N/A,consensus,high,pro-écologie,techniquement il n'y a pas de résidus si vous ...,N/A
2779,Cette allégation est préoccupante car elle pou...,Environ un mètre de profondeur des sols n’a pa...,"Dans un rapport d'inspection des ouvrages, il ...",N/A,facts,high,pro-écologie,selon les sondages effectués par martin de la ...,Martin de la société Geo Tech
...,...,...,...,...,...,...,...,...,...
56780,Cette allégation nécessite une vérification ap...,Le projet ne fixe plus aucun objectif commun d...,Dans le même contexte de discussion sur les én...,N/A,facts,high,pro-écologie,Le projet ne fixe plus aucun objectif commun d...,N/A
56972,"Cette déclaration, si elle est avérée, pourrai...",Les pertes de récoltes atteignent plus de 80% ...,L'agriculteur déclare avoir subi plus de 80% d...,N/A,facts,high,pro-écologie,vous dépasser les quatre-vingts pourcent de pe...,"Christian Taillant, agriculteur"
58129,L'assertion d'une explosion de cancers est une...,Il y a une explosion de cancers liés à la poll...,L'avocat mentionne une augmentation des cas de...,N/A,consensus,high,pro-écologie,Il y a une explosion de cancers cancers voilà.,"Fiodor Rilov, avocat au barreau de Paris"
59791,Bien que le changement climatique puisse entra...,Les températures sont six degrés au-dessus des...,Un intervenant mentionne des chiffres alarmant...,N/A,facts,high,pro-écologie,les températures six sont degrés quatre au-des...,N/A


In [139]:
# keep only interesting claims
def disinformation_score(row):
    try:
        return 'high' in [row.claims[i]['disinformation_score'] for i in range(len(row))]
    except:
        return [False]

score = df.apply(lambda row: disinformation_score(row), axis=1)

In [143]:
df['claims'].apply(lambda row: row[0]['claim'])

IndexError: index 0 is out of bounds for axis 0 with size 0

In [144]:
score

0        [False]
1        [False]
2        [False]
3        [False]
4        [False]
          ...   
34514    [False]
34515    [False]
34516    [False]
34517    [False]
34518    [False]
Length: 34519, dtype: object

In [141]:
score[score.apply(lambda row: 'high' in row)]

Series([], dtype: object)

In [130]:
df = df[score=='high']

,id,start,text,channel_name,channel_is_radio,channel_program_type,channel_program,themes,keywords,num_keywords,num_tokens,claims
39,365a111750a50eaf9dc43c40d3b1c3cf01610dc9b8a41d...,2023-09-03 08:10:00,porte-monnaie avec la forêt vous profitez d'un...,europe1,True,Information - Magazine,Europe 1 Matin Week-end,"[""biodiversite_causes"", ""adaptation_climatique...","[{""keyword"": ""surconsommation"", ""timestamp"": 1...",2,516,[{'analysis': 'Cette allégation semble perpétu...
336,1d7b33cc8904908a90e97c78333af9e8a636609a8235f2...,2023-09-10 07:30:00,sur europe un lors d'un nouveau journal bonjou...,europe1,True,Information - Magazine,Europe 1 Matin Week-end,"[""biodiversite_concepts_generaux"", ""changement...","[{""keyword"": ""terre"", ""timestamp"": 16943238890...",2,521,[{'analysis': 'Il n'existe pas de consensus sc...
522,6397026d01262adc2b42d90d5f121813682055fc9a11de...,2023-09-14 18:12:00,migrants l'origine du sel fin du burkina faso ...,europe1,True,Information - Magazine,Punchline,"[""biodiversite_concepts_generaux"", ""ressources...","[{""keyword"": ""\u00e9cologique"", ""timestamp"": 1...",2,618,[{'analysis': 'Cette allégation semble infondé...
544,bbd54614cfd2b9a7244f0dd9b27c8e8d1d4b69819d68c6...,2023-09-15 07:14:00,forcément désespérée pour l'humanité en l'occu...,europe1,True,Information - Magazine,Europe 1 Matin,"[""biodiversite_concepts_generaux"", ""ressources...","[{""keyword"": ""\u00e9cologie"", ""timestamp"": 169...",5,544,[{'analysis': 'Cette allégation minimise la gr...
595,34a41bf34b35ee91fc147601fb8c21a366a2f568b9060b...,2023-09-15 19:30:00,jusqu'au trente septembre détaille sofia au qu...,europe1,True,Information - Magazine,Soir,"[""biodiversite_concepts_generaux"", ""ressources...","[{""keyword"": ""terre"", ""timestamp"": 16947991040...",2,533,[{'analysis': 'L'utilisation de l'expression '...
...,...,...,...,...,...,...,...,...,...,...,...,...
34238,850495959eae4bdf95e26192152e2d65f75aee86a7f632...,2024-08-16 09:14:00,environnements sonores je peux vous donner un ...,sud-radio,True,Information - Magazine,Le Grand Matin,"[""ressources_solutions"", ""attenuation_climatiq...","[{""keyword"": ""bon pour la plan\u00e8te"", ""time...",6,555,[{'analysis': 'Cette allégation est trompeuse....
34277,963ec2c22db26525473e4b603480e7d2388f17c8536b9f...,2024-08-20 10:04:00,dans la nuit pierre à frontignan dans l'héraul...,sud-radio,True,Information - Magazine,Le Grand Matin,"[""ressources_indirectes"", ""attenuation_climati...","[{""keyword"": ""feux de for\u00eat"", ""timestamp""...",1,488,[{'analysis': 'Cette allégation semble peu fon...
34500,16e6ae8580f822882631c8092ec13ee748e65b2ebc3b7c...,2024-09-10 13:30:00,commentaires mais euh beaucoup d'historien rap...,sud-radio,True,Information - Magazine,Sud Radio dans tous ses états,"[""attenuation_climatique_solutions"", ""ressourc...","[{""keyword"": ""panneaux solaires"", ""timestamp"":...",1,541,[{'analysis': 'Cette allégation semble exploit...
34505,e33f6d58f4bde52cd98263dcfab9f189b4d5c1d4648a0a...,2024-09-11 07:42:00,a dit à donald trump à plusieurs reprises au m...,sud-radio,True,Information - Magazine,Le Grand Matin,"[""changement_climatique_constat""]","[{""keyword"": ""climatique"", ""timestamp"": 172603...",1,493,[{'analysis': 'Cette affirmation nie le consen...


In [129]:
score.unique()

array(['low', 'medium', 'very low', False, 'high', 'N/A', 'Medium'],
      dtype=object)

In [38]:
df.iloc[0].claims[0].keys()

dict_keys(['analysis', 'claim', 'context', 'contradiction', 'disinformation_category', 'disinformation_score', 'pro_anti', 'quote', 'speaker'])

In [14]:
df.iloc[1].claims[0]['claim']

"Les propriétaires de campings peuvent réaliser jusqu'à 50% d'économie d'eau en installant des douches connectées."

In [15]:
df.iloc[1].claims[0]['quote']

"la facture s'allège de manière non négligeable avec cinquante pour cent d'économie d'eau atteignant parfois les six mille euros annuel"

- 1 an dont canicule cop et crise agricole
- few shots
- rag avec la taxonomie cards entre les claims extraites et les cards 
- liste de claims comment peut on les classifier 
- chaine 1 mois de contenu > propos fallacieux > on clique sur 1 > on voit la classification 
labeliser 

In [110]:
sample = df.iloc[:10]

def extract_column(row, column):
    try:
        return row[0][column]
    except:
        return ''

claims = sample['claims'].apply(lambda row: extract_column(row, 'quote'))
result, n_tokens = classify_claims(claims)
sample = pd.merge(sample, result, left_index=True, right_index=True)

In [111]:
sample

,id,start,text,channel_name,channel_is_radio,channel_program_type,channel_program,themes,keywords,num_keywords,num_tokens,claims,cards_label_pred
0,1dcd4b454f8bac42440259ce26a1a2192051186bb5728b...,2023-09-01 06:26:00,<unk> <unk> <unk> <unk> aerosmith en janvier m...,europe1,True,Information - Magazine,Bonjour,"[""biodiversite_concepts_generaux"", ""adaptation...","[{""keyword"": ""eaux"", ""timestamp"": 169354246806...",1,383,[{'analysis': 'L'allégation semble promouvoir ...,0_accepted
1,0eb5805fa23e0819f817ea10fe1fccd19e61e40a1239cc...,2023-09-01 06:50:00,la très grande majorité d'entre eux ne connais...,europe1,True,Information - Magazine,Bonjour,"[""biodiversite_concepts_generaux"", ""changement...","[{""keyword"": ""eaux"", ""timestamp"": 169354380309...",3,593,[{'analysis': 'Cette affirmation est généralem...,0_accepted
2,b6d54aefb250671e7754a688411ce9e68badcad88a665b...,2023-09-01 07:38:00,mais titeuf ne vieillit pas le monde change ti...,europe1,True,Information - Magazine,Europe 1 Matin,"[""changement_climatique_constat""]","[{""keyword"": ""bilan carbone"", ""timestamp"": 169...",1,606,[{'analysis': 'L'allégation sur le fait que Ti...,0_accepted
3,23c2d3b292d9ab0fb0d0b2b8c34f3b88708c79ffab2d66...,2023-09-01 08:44:00,dû travailler très vite le journal arrive à no...,europe1,True,Information - Magazine,Europe 1 Matin,"[""biodiversite_concepts_generaux"", ""ressources...","[{""keyword"": ""eaux"", ""timestamp"": 169355074908...",3,515,[{'analysis': 'L'affirmation que le nucléaire ...,0_accepted
4,71df0ce2b34afa23391d8e31d35ccd213ae2a881b7ce41...,2023-09-01 08:46:00,pas avoir lieu ni même européens existait ni l...,europe1,True,Information - Magazine,Europe 1 Matin,"[""biodiversite_concepts_generaux"", ""changement...","[{""keyword"": ""\u00e9cologiste"", ""timestamp"": 1...",2,536,[{'analysis': 'Cette allégation souligne un év...,0_accepted
5,2ea97e4eaf052c5bb3d2c3d6537d107904643fd9840d72...,2023-09-01 11:30:00,petits plats de laurent mariotte le magazine c...,europe1,True,Information - Magazine,Pascal Praud et vous,"[""changement_climatique_causes"", ""ressources"",...","[{""keyword"": ""surconsommation"", ""timestamp"": 1...",1,519,[{'analysis': 'Cette allégation est généraleme...,0_accepted
6,7b39c46fdaf5c1ed5aef83f4045bfd27e15bc72f79d0aa...,2023-09-01 11:44:00,des super produits et des supers prix découvre...,europe1,True,Information - Magazine,Pascal Praud et vous,"[""biodiversite_concepts_generaux"", ""biodiversi...","[{""keyword"": ""surconsommation"", ""timestamp"": 1...",4,581,[{'analysis': 'Cette alliégation est effective...,1_its_not_happening
7,f58736b7504a325a05797d4182048a9b64a8ed34408cb6...,2023-09-01 12:02:00,à la gare sncf d'orange tout près du lycée pro...,europe1,True,Information - Magazine,Pascal Praud et vous,"[""biodiversite_concepts_generaux"", ""changement...","[{""keyword"": ""gaz \u00e0 effet de serre"", ""tim...",2,511,[{'analysis': 'Cette allégation est conforme a...,0_accepted
8,5203d9251c9846284a9c3160c2d5276ee82b291f9ea506...,2023-09-01 13:12:00,été en manque de braderie donc cette année on ...,europe1,True,Information - Magazine,Europe 1 Matin,"[""ressources"", ""changement_climatique_constat""]","[{""keyword"": ""bilan carbone"", ""timestamp"": 169...",1,574,[{'analysis': 'Bien que le personnage de titeu...,0_accepted


In [112]:
for claim, row in zip(claims, sample.iterrows()):
    pprint.pprint(claim)
    pprint.pprint(row[1]['cards_label_pred'])
    print(" ")

"les initiatives positives de jean z une nova sion pour économiser l'eau"
'0_accepted'
 
("la facture s'allège de manière non négligeable avec cinquante pour cent "
 "d'économie d'eau atteignant parfois les six mille euros annuel")
'0_accepted'
 
("j'ai bien aimé dessinées titeuf qui explique ce qu'est le bilan carbone "
 'comment ça marche avec sa grammaire enfantine.')
'0_accepted'
 
('pour la cgt le nucléaire fait partie du mix énergétique et demeure '
 'indispensable pour lutter contre le dérèglement climatique.')
'0_accepted'
 
("La population ou de nos ancêtres prédécesseurs d'homo sapiens aurait alors "
 "chuté à mille trois cents individus à cause d'un dérèglement climatique.")
'0_accepted'
 
'une voiture bien entretenue peut éviter une surconsommation de carburant'
'0_accepted'
 
'Les moules sont bien françaises, bien iodées, bien charnues.'
'1_its_not_happening'
 
('Les émissions de gaz à effet de serre provoquent des vagues de chaleur de '
 'plus en plus intenses et durable.

In [86]:
# check quote 
test = sample
#test['claim'] =  sample['claims'].apply(lambda row: extract_column(row, 'claim'))
test['quote'] =  sample['claims'].apply(lambda row: extract_column(row, 'quote'))

/var/folders/qs/h69xc93902zdpt61kf3m_p1r0000gn/T/ipykernel_1791/4253958984.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['claim'] =  sample['claims'].apply(lambda row: extract_column(row, 'claim'))
/var/folders/qs/h69xc93902zdpt61kf3m_p1r0000gn/T/ipykernel_1791/4253958984.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['quote'] =  sample['claims'].apply(lambda row: extract_column(row, 'quote'))


In [89]:
test

,id,start,text,channel_name,channel_is_radio,channel_program_type,channel_program,themes,keywords,num_keywords,num_tokens,claims,claim,quote
0,1dcd4b454f8bac42440259ce26a1a2192051186bb5728b...,2023-09-01 06:26:00,<unk> <unk> <unk> <unk> aerosmith en janvier m...,europe1,True,Information - Magazine,Bonjour,"[""biodiversite_concepts_generaux"", ""adaptation...","[{""keyword"": ""eaux"", ""timestamp"": 169354246806...",1,383,[{'analysis': 'L'allégation semble promouvoir ...,les initiatives positives de jean z une nova s...,les initiatives positives de jean z une nova s...
1,0eb5805fa23e0819f817ea10fe1fccd19e61e40a1239cc...,2023-09-01 06:50:00,la très grande majorité d'entre eux ne connais...,europe1,True,Information - Magazine,Bonjour,"[""biodiversite_concepts_generaux"", ""changement...","[{""keyword"": ""eaux"", ""timestamp"": 169354380309...",3,593,[{'analysis': 'Cette affirmation est généralem...,Les propriétaires de campings peuvent réaliser...,la facture s'allège de manière non négligeable...
2,b6d54aefb250671e7754a688411ce9e68badcad88a665b...,2023-09-01 07:38:00,mais titeuf ne vieillit pas le monde change ti...,europe1,True,Information - Magazine,Europe 1 Matin,"[""changement_climatique_constat""]","[{""keyword"": ""bilan carbone"", ""timestamp"": 169...",1,606,[{'analysis': 'L'allégation sur le fait que Ti...,Titeuf explique ce qu'est le bilan carbone.,j'ai bien aimé dessinées titeuf qui explique c...
3,23c2d3b292d9ab0fb0d0b2b8c34f3b88708c79ffab2d66...,2023-09-01 08:44:00,dû travailler très vite le journal arrive à no...,europe1,True,Information - Magazine,Europe 1 Matin,"[""biodiversite_concepts_generaux"", ""ressources...","[{""keyword"": ""eaux"", ""timestamp"": 169355074908...",3,515,[{'analysis': 'L'affirmation que le nucléaire ...,Le nucléaire fait partie du mix énergétique et...,pour la cgt le nucléaire fait partie du mix én...
4,71df0ce2b34afa23391d8e31d35ccd213ae2a881b7ce41...,2023-09-01 08:46:00,pas avoir lieu ni même européens existait ni l...,europe1,True,Information - Magazine,Europe 1 Matin,"[""biodiversite_concepts_generaux"", ""changement...","[{""keyword"": ""\u00e9cologiste"", ""timestamp"": 1...",2,536,[{'analysis': 'Cette allégation souligne un év...,"Il y a neuf cent mille ans, la population de n...",La population ou de nos ancêtres prédécesseurs...
5,2ea97e4eaf052c5bb3d2c3d6537d107904643fd9840d72...,2023-09-01 11:30:00,petits plats de laurent mariotte le magazine c...,europe1,True,Information - Magazine,Pascal Praud et vous,"[""changement_climatique_causes"", ""ressources"",...","[{""keyword"": ""surconsommation"", ""timestamp"": 1...",1,519,[{'analysis': 'Cette allégation est généraleme...,une voiture bien entretenue peut éviter une su...,une voiture bien entretenue peut éviter une su...
6,7b39c46fdaf5c1ed5aef83f4045bfd27e15bc72f79d0aa...,2023-09-01 11:44:00,des super produits et des supers prix découvre...,europe1,True,Information - Magazine,Pascal Praud et vous,"[""biodiversite_concepts_generaux"", ""biodiversi...","[{""keyword"": ""surconsommation"", ""timestamp"": 1...",4,581,[{'analysis': 'Cette alliégation est effective...,"Les moules sont bien françaises, bien iodées e...","Les moules sont bien françaises, bien iodées, ..."
7,f58736b7504a325a05797d4182048a9b64a8ed34408cb6...,2023-09-01 12:02:00,à la gare sncf d'orange tout près du lycée pro...,europe1,True,Information - Magazine,Pascal Praud et vous,"[""biodiversite_concepts_generaux"", ""changement...","[{""keyword"": ""gaz \u00e0 effet de serre"", ""tim...",2,511,[{'analysis': 'Cette allégation est conforme a...,Les émissions de gaz à effet de serre provoque...,Les émissions de gaz à effet de serre provoque...
8,5203d9251c9846284a9c3160c2d5276ee82b291f9ea506...,2023-09-01 13:12:00,été en manque de braderie donc cette année on ...,europe1,True,Information - Magazine,Europe 1 Matin,"[""ressources"", ""changement_climatique_constat""]","[{""keyword"": ""bilan carbone"", ""timestamp"": 169...",1,574,[{'analysis': 'Bien que le personnage de tite

In [101]:
def quote_in_text(row):
    try:
        return row['quote'] in row['text']
    except:
        return False
test['quote_in_text'] = test.apply(lambda row: quote_in_text(row))

/var/folders/qs/h69xc93902zdpt61kf3m_p1r0000gn/T/ipykernel_1791/3870671050.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['quote_in_text'] = test.apply(lambda row: quote_in_text(row))


In [104]:
test.apply(lambda row: quote_in_text(row), axis=1)

0     True
1     True
2    False
3    False
4    False
5    False
6    False
7    False
8     True
9     True
dtype: bool

In [105]:
test['quote'].iloc[2]

"j'ai bien aimé dessinées titeuf qui explique ce qu'est le bilan carbone comment ça marche avec sa grammaire enfantine."

In [106]:
pprint.pprint(test['text'].iloc[2])

('mais titeuf ne vieillit pas le monde change titeuf a toujours le même âge '
 'sont drame on peut lui rêverait de devenir un adulte toujours entouré de ses '
 'parents sa bande de copains et nadia son amoureux inaccessible zep son '
 "auteur ne s'en lasse pas et nous non plus que c'est mon rendez-vous on fait "
 "avec l'enfance quand je pars sur un album de titeuf dont je suis content de "
 'pouvoir ouvrir les vannes et puis tester une autre titeuf reste un prix '
 "adolescent alerte sur le monde qui l'entoure à la de récré nouveaux de "
 'sujets nouveaux de sujets discussion de comme discussion la comme crise la '
 "climatique crise ou climatique le consentement sexuel moi j'aime bien quand "
 "il y a des sujets un petit peu casse gueule donc j'ai bien aimé dessinées "
 "titeuf qui explique ce qu'est le bilan carbone comment ça marche avec sa "
 "grammaire enfantine j'aime bien jouer avec cette réalité enfantine m'amuser "
 "j'aime bien cette idée que les enfants s'insultent zizi mai

In [78]:
sample = df.iloc[:10]

In [70]:
df.shape

(34519, 12)

In [107]:
sample

,id,start,text,channel_name,channel_is_radio,channel_program_type,channel_program,themes,keywords,num_keywords,num_tokens,claims,claim,quote,quote_in_text
0,1dcd4b454f8bac42440259ce26a1a2192051186bb5728b...,2023-09-01 06:26:00,<unk> <unk> <unk> <unk> aerosmith en janvier m...,europe1,True,Information - Magazine,Bonjour,"[""biodiversite_concepts_generaux"", ""adaptation...","[{""keyword"": ""eaux"", ""timestamp"": 169354246806...",1,383,[{'analysis': 'L'allégation semble promouvoir ...,les initiatives positives de jean z une nova s...,les initiatives positives de jean z une nova s...,NaN
1,0eb5805fa23e0819f817ea10fe1fccd19e61e40a1239cc...,2023-09-01 06:50:00,la très grande majorité d'entre eux ne connais...,europe1,True,Information - Magazine,Bonjour,"[""biodiversite_concepts_generaux"", ""changement...","[{""keyword"": ""eaux"", ""timestamp"": 169354380309...",3,593,[{'analysis': 'Cette affirmation est généralem...,Les propriétaires de campings peuvent réaliser...,la facture s'allège de manière non négligeable...,NaN
2,b6d54aefb250671e7754a688411ce9e68badcad88a665b...,2023-09-01 07:38:00,mais titeuf ne vieillit pas le monde change ti...,europe1,True,Information - Magazine,Europe 1 Matin,"[""changement_climatique_constat""]","[{""keyword"": ""bilan carbone"", ""timestamp"": 169...",1,606,[{'analysis': 'L'allégation sur le fait que Ti...,Titeuf explique ce qu'est le bilan carbone.,j'ai bien aimé dessinées titeuf qui explique c...,NaN
3,23c2d3b292d9ab0fb0d0b2b8c34f3b88708c79ffab2d66...,2023-09-01 08:44:00,dû travailler très vite le journal arrive à no...,europe1,True,Information - Magazine,Europe 1 Matin,"[""biodiversite_concepts_generaux"", ""ressources...","[{""keyword"": ""eaux"", ""timestamp"": 169355074908...",3,515,[{'analysis': 'L'affirmation que le nucléaire ...,Le nucléaire fait partie du mix énergétique et...,pour la cgt le nucléaire fait partie du mix én...,NaN
4,71df0ce2b34afa23391d8e31d35ccd213ae2a881b7ce41...,2023-09-01 08:46:00,pas avoir lieu ni même européens existait ni l...,europe1,True,Information - Magazine,Europe 1 Matin,"[""biodiversite_concepts_generaux"", ""changement...","[{""keyword"": ""\u00e9cologiste"", ""timestamp"": 1...",2,536,[{'analysis': 'Cette allégation souligne un év...,"Il y a neuf cent mille ans, la population de n...",La population ou de nos ancêtres prédécesseurs...,NaN
5,2ea97e4eaf052c5bb3d2c3d6537d107904643fd9840d72...,2023-09-01 11:30:00,petits plats de laurent mariotte le magazine c...,europe1,True,Information - Magazine,Pascal Praud et vous,"[""changement_climatique_causes"", ""ressources"",...","[{""keyword"": ""surconsommation"", ""timestamp"": 1...",1,519,[{'analysis': 'Cette allégation est généraleme...,une voiture bien entretenue peut éviter une su...,une voiture bien entretenue peut éviter une su...,NaN
6,7b39c46fdaf5c1ed5aef83f4045bfd27e15bc72f79d0aa...,2023-09-01 11:44:00,des super produits et des supers prix découvre...,europe1,True,Information - Magazine,Pascal Praud et vous,"[""biodiversite_concepts_generaux"", ""biodiversi...","[{""keyword"": ""surconsommation"", ""timestamp"": 1...",4,581,[{'analysis': 'Cette alliégation est effective...,"Les moules sont bien françaises, bien iodées e...","Les moules sont bien françaises, bien iodées, ...",NaN
7,f58736b7504a325a05797d4182048a9b64a8ed34408cb6...,2023-09-01 12:02:00,à la gare sncf d'orange tout près du lycée pro...,europe1,True,Information - Magazine,Pascal Praud et vous,"[""biodiversite_concepts_generaux"", ""changement...","[{""keyword"": ""gaz \u00e0 effet de serre"", ""tim...",2,511,[{'analysis': 'Cette allégation est conforme a...,Les émissions de gaz à effet de serre provoque...,Les émissions de gaz à effet de serre provoque...,NaN
8,5203d9251c9846284a9c3160c2d5276ee82b291f9ea506...,2023-09-01 13:12:00,été en manque de braderie donc cette année on ...,europe1,True,Information - Magazine,Europe 1 Matin,"[""ressources"", ""changement_climatique_constat""]","[{""keyword"": ""bilan carbone"", ""timestamp"": 169...",1,574

In [30]:
sample['cards_label_pred'] = result.values

ValueError: Length of values (11) does not match length of index (10)

In [ ]:
# classification ARCOM 